In [1]:
# Install the requirements in Google Colab
#!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainerCallback, Trainer, pipeline
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch


device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_path = "./checkpoint-116000"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_path
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_path)

# Set up the chat format
#model, tokenizer = setup_chat_format(model=None, tokenizer=tokenizer)


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [3]:
# Let's test the base model before training
prompt = "Thu nhập từ kinh doanh của cá nhân có được giảm trừ gia cảnh không?"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=1024, do_sample=True)
# Decode and print the output
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Before training:
system
You are a helpful assistant.
user
Thu nhập từ kinh doanh của cá nhân có được giảm trừ gia cảnh không?
Thu nhập từ kinh doanh của cá nhân có được giữ lại không?ervation/
Thu nhập từ kinh doanh của cá nhân có giảm thêm vào số liệu từ ngân hàng không?	Run
Thu nhập từ tài sản của cá nhân có giảm thêm vào số liệu từ cơ quan thuế không?	Check
Thu nhập trung bình hằng tháng của cá nhân không giảm là thu được từ các nguồn kinh
dodoanh do cá nhân tư nhân, các công ty hành chính, các hộ tư nhân, cá nhân trung chuyển và
các nhà môi giới chuyển nhượng đầu tư cá nhân được hưởng thu nhập từ hoạt động sản xuất
kinh doanh.
14321
14322
14323
14324
14325
Thu nhập trung bình hằng tháng của nhóm làm việc trước kinh doanh của cá
nhân không giảm?
Thu nhập
thì được giảm bao nhiêu %?
vào số liệu từ ngân hàng
Thu nhập từ tài sản của cá nhân là được giữ lại vì làm tăng thu nhập từ kinh
Thu nhập từ việc làm thuê của cá nhân (thu nhập tiền công) giảm bao nhiêu %?
ngay
thu nhập đó
Thời gian

In [8]:
def load_json_dataset(train_path, val_path):
    dataset = load_dataset('json', data_files={
        'train': train_path,
        'validation': val_path
    })
    return dataset['train'], dataset['validation']

# Load and train CoLA

In [11]:
train_path = r"ViGLUE_Law\cola\train_phi4.json"
val_path = r"ViGLUE_Law\cola\validation_phi4.json"

train_dataset, val_dataset = load_json_dataset(train_path, val_path)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 262
Validation dataset size: 37


In [15]:
def formatting_prompt_func_COLA(example):
        return f"<Instruct>Is this example grammatically correct and sensible?</Instruct>\n<Format>Sentence format</Format>\n<Question>{example['sentence']}</Question>\n<Answer>{example['label']}</Answer>"

In [ ]:

# Configure the SFTTrainer
training_args = SFTConfig(
    output_dir="./sft_output_CoLA",
    num_train_epochs=50,  # Set the number of epochs here
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    eval_strategy="epoch",
    eval_steps=5,
)

In [34]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=formatting_prompt_func_COLA,)

trainer.train()

metrics = trainer.evaluate()

print(metrics)

Epoch,Training Loss,Validation Loss
1,0.162200,1.282118
2,0.158100,1.408831
3,0.153400,1.327056
4,0.144900,1.377716
5,0.147100,1.381655
6,0.141100,1.460059
7,0.135500,1.389682
8,0.124000,1.506143
9,0.131000,1.444576
10,0.137300,1.489828


{'eval_loss': 1.6701245307922363, 'eval_runtime': 0.2486, 'eval_samples_per_second': 148.836, 'eval_steps_per_second': 20.113}


In [ ]:

# Load the inference pipeline
inference = pipeline("text-generation", model=trainer.model, tokenizer=trainer.tokenizer, device=0)  # use device=0 for GPU

def evaluate_pipeline_on_val(val_dataset, max_samples=10):
    results = []

    for i, example in enumerate(val_dataset):
        if i >= max_samples:
            break

        prompt = f"<Instruct>Is this example grammatically correct and sensible?</Instruct>\n" \
                 f"<Format>Sentence format</Format>\n" \
                 f"<Question>{example['sentence']}</Question>\n<Answer>"

        output = inference(prompt, max_new_tokens=10)[0]['generated_text']
        predicted = output.split("<Answer>")[-1].strip()

        results.append({
            "input": example["sentence"],
            "true_label": example["label"],
            "predicted": predicted
        })

    return results


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Device set to use cuda:0


In [57]:
results = evaluate_pipeline_on_val(val_dataset)
for res in results:
    print(f"Sentence: {res['input']}")
    print(f"True Label: {res['true_label']}")
    print(f"Predicted: {res['predicted']}\n")

Sentence: Thị trưởng coi đề xuất xây dựng vỉa hè từ Dartmouth đến Smith là vô lý.
True Label: unacceptable
Predicted: acceptable</Answer>

Sentence: Wind đã âm mưu đàm phán để chấm dứt chiến tranh ở Việt Nam trong danh dự.
True Label: unacceptable
Predicted: acceptable</Answer>

Sentence: Người thợ kim hoàn viết nguệch ngoạc bản hợp đồng có tên anh ta.
True Label: unacceptable
Predicted: acceptable</Answer>

Sentence: Mụ phù thủy đã đầu độc bọn trẻ.
True Label: acceptable
Predicted: acceptable</Answer>

Sentence: viên cảnh sát đã gặp ở công viên phải không?
True Label: acceptable
Predicted: acceptable</Answer>

Sentence: John gọi tổng thống là kẻ ngốc.
True Label: acceptable
Predicted: acceptable</Answer>

Sentence: Nó cố gắng rời khỏi đất nước.
True Label: unacceptable
Predicted: unacceptable</Answer>

Sentence: Có vẻ như ai đó đã đánh nạn nhân không rõ danh tính vào lúc sáng sớm.
True Label: acceptable
Predicted: acceptable</Answer>

Sentence: Colin hỏi liệu họ có thể vay thế chấp đư

In [62]:
def compute_accuracy(results):
    correct = 0
    total = len(results)

    for res in results:
        # Normalize prediction: strip, lowercase, and remove trailing tokens
        pred = res["predicted"].lower().replace("</answer>", "").strip()
        true = res["true_label"].lower().strip()

        if pred == true:
            correct += 1

    accuracy = correct / total if total > 0 else 0
    return accuracy

In [63]:
import pandas as pd

# Compute accuracy
accuracy = compute_accuracy(results)

# Create initial DataFrame with CoLA accuracy
df_results = pd.DataFrame([{
    "task": "CoLA",
    "accuracy": accuracy
}])

print(df_results)


   task  accuracy
0  CoLA       0.7


# Load & Train MNLI

In [64]:
train_path = r"ViGLUE_Law\mnli\train_phi4.json"
val_path = r"ViGLUE_Law\mnli\validation_matched_phi4.json"

train_dataset, val_dataset = load_json_dataset(train_path, val_path)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 9216
Validation dataset size: 222


In [65]:
def formatting_prompt_func_MNLI(example):
    return (
        "<Instruct>Given a premise and a hypothesis, determine whether the hypothesis is true, false, or inconclusive based on the premise.</Instruct>\n"
        "<Format>Premise and Hypothesis format</Format>\n"
        f"<Premise>{example['premise']}</Premise>\n"
        f"<Hypothesis>{example['hypothesis']}</Hypothesis>\n"
        f"<Answer>{example['label']}</Answer>"
    )


In [67]:
# Configure the SFTTrainer
training_args = SFTConfig(
    output_dir="./sft_output_MNLI",
    num_train_epochs=3,  # Set the number of epochs here
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    eval_strategy="epoch",
    eval_steps=5,
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=formatting_prompt_func_MNLI,
    )

trainer.train()

metrics = trainer.evaluate()

print(metrics)

Epoch,Training Loss,Validation Loss
1,0.613800,1.105324
2,0.473000,1.252992
3,0.184000,1.425677


{'eval_loss': 1.4256772994995117, 'eval_runtime': 64.7777, 'eval_samples_per_second': 3.427, 'eval_steps_per_second': 0.432}


In [74]:
# Load the inference pipeline
inference = pipeline(
    "text-generation",
    model=trainer.model,
    tokenizer=trainer.tokenizer,
    device=0  # GPU
)

def evaluate_pipeline_on_mnli(val_dataset, max_samples=10):
    results = []

    for i, example in enumerate(val_dataset):
        if i >= max_samples:
            break

        premise = example["premise"]
        hypothesis = example["hypothesis"]
        # Build an MNLI prompt
        prompt = (
            "<Instruct>Given a premise and a hypothesis, decide if the hypothesis is true, false, or inconclusive based on the premise.</Instruct>\n"
            "<Format>Premise and Hypothesis format</Format>\n"
            f"<Premise>{premise}</Premise>\n"
            f"<Hypothesis>{hypothesis}</Hypothesis>\n"
            "<Answer>"
        )

        # Generate and extract the model’s answer
        output = inference(prompt, max_new_tokens=10)[0]["generated_text"]
        # Anything after the last <Answer> tag is our prediction
        predicted = output.split("<Answer>")[-1].strip().replace("</Answer>", "").lower()

        results.append({
            "premise": premise,
            "hypothesis": hypothesis,
            "true_label": example["label"].lower(),
            "predicted": predicted
        })

    return results

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Device set to use cuda:0


In [77]:
mnli_results = evaluate_pipeline_on_mnli(val_dataset, max_samples=1000)
for res in mnli_results:
    print(f"Premise: {res['premise']}")
    print(f"Hypothesis: {res['hypothesis']}")
    print(f"True Label: {res['true_label']}")
    print(f"Predicted: {res['predicted']}\n")

Premise: và thực tế là anh ấy là kẻ trốn quân dịch
Hypothesis: Họ đã trốn tránh quân dịch, tôi sẽ cho bạn biết.
True Label: entailment
Predicted: neutral

Premise: Quy tắc cuối cùng bao gồm Đánh giá chủ nghĩa liên bang theo Sắc lệnh hành pháp
Hypothesis: Quy tắc cuối cùng có đánh giá về chủ nghĩa liên bang đã được Tổng thống bổ sung thông qua lệnh hành pháp.
True Label: neutral
Predicted: neutral

Premise: Thông lệ chung của Bộ Quốc phòng để quản lý môi trường này là tạo ra những nỗ lực tích cực giảm thiểu rủi ro trong các chương trình của mình.
Hypothesis: Tạo ra những nỗ lực giảm thiểu rủi ro là thông lệ.
True Label: entailment
Predicted: entailment

Premise: Nếu bạn đến bằng thuyền, bạn có thể sử dụng bãi biển Caravelle; nếu không bạn sẽ phải trả một khoản phí danh nghĩa cho câu lạc bộ nghỉ dưỡng sở hữu diện tích đó.
Hypothesis: Những người đến bãi biển Caravelle bằng đường bộ phải trả phí.
True Label: entailment
Predicted: entailment

Premise: Màn hình chính phải được tích hợp vào 

In [78]:
# Suppose you just computed MNLI accuracy
mnli_accuracy = compute_accuracy(mnli_results)

# Append MNLI result to df_results
df_results.loc[len(df_results)] = {
    "task": "MNLI",
    "accuracy": mnli_accuracy
}

print(df_results)

   task  accuracy
0  CoLA  0.700000
1  MNLI  0.716216


# Load & Train MRPC

In [79]:
train_path = r"ViGLUE_Law\mrpc\train_phi4.json"
val_path = r"ViGLUE_Law\mrpc\validation_phi4.json"

train_dataset, val_dataset = load_json_dataset(train_path, val_path)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 159
Validation dataset size: 17


In [ ]:
def formatting_prompt_func_MRPC(example):
    return (
        "<Instruct>Do the two sentences mean the same thing (answer yes or no)?</Instruct>\n"
        "<Format>Sentence pair format</Format>\n"
        f"<Question>Sentence 1: {example['sentence1']}\nSentence 2: {example['sentence2']}</Question>\n"
        f"<Answer>{example['label']}</Answer>"
    )

In [98]:
training_args = SFTConfig(
    output_dir="./sft_output_MRPC",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    logging_steps=10,
    eval_strategy="epoch",
    #save_strategy="steps",               # Save model every 'n' steps instead of epochs
    #save_steps=500,                       # Save every 500 steps (adjust as needed)
    save_total_limit=2,                   # Keep only the last 2 checkpoints
    #load_best_model_at_end=True,          # Load the best model at the end based on eval loss or accuracy
    metric_for_best_model="eval_loss",    # Track accuracy for best model selection
    greater_is_better=True               # Higher accuracy is better
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=formatting_prompt_func_MRPC,
    )

trainer.train()

metrics = trainer.evaluate()

print(metrics)

Epoch,Training Loss,Validation Loss
1,0.068100,2.299452
2,0.061000,2.509733
3,0.058300,2.556475
4,0.057300,2.513298
5,0.056200,2.489810
6,0.054000,2.484581
7,0.053600,2.491024
8,0.053100,2.496912
9,0.052700,2.501697
10,0.052400,2.502940


{'eval_loss': 2.5029399394989014, 'eval_runtime': 0.6733, 'eval_samples_per_second': 25.248, 'eval_steps_per_second': 4.456}


In [99]:
def evaluate_pipeline_on_mrpc(val_dataset, max_samples=10):
    results = []

    for i, example in enumerate(val_dataset):
        if i >= max_samples:
            break

        sentence1 = example["sentence1"]
        sentence2 = example["sentence2"]

        prompt = (
            "<Instruct>Do the two sentences mean the same thing (answer yes or no)?</Instruct>\n"
            "<Format>Sentence pair format</Format>\n"
            f"<Question>Sentence 1: {example['sentence1']}\nSentence 2: {example['sentence2']}</Question>\n"
            f"<Answer>"
        )

        output = inference(prompt, max_new_tokens=10)[0]['generated_text']
        predicted_raw = output.split("Answer:")[-1].strip().lower()

        # Normalize to MRPC string label
        if "yes" in predicted_raw:
            predicted = "paraphrase"
        elif "no" in predicted_raw:
            predicted = "not_paraphrase"
        else:
            predicted = "unknown"

        true_label = example["label"]
        true_label_str = "paraphrase" if true_label in [1, "1", "paraphrase"] else "not_paraphrase"

        results.append({
            "sentence1": sentence1,
            "sentence2": sentence2,
            "true_label": true_label_str,
            "predicted": predicted
        })

    return results

mrpc_results = evaluate_pipeline_on_mrpc(val_dataset, max_samples=1000)
for res in mrpc_results:
    print(f"Sentence 1: {res['sentence1']}")
    print(f"Sentence 2: {res['sentence2']}")
    print(f"True Label: {res['true_label']}")
    print(f"Predicted: {res['predicted']}\n")

Sentence 1: Chưa có ngày nào được ấn định cho phiên tòa dân sự hoặc hình sự.
Sentence 2: Chưa có ngày ấn định cho các vụ án hình sự hay dân sự, nhưng Shanley đã không nhận tội.
True Label: not_paraphrase
Predicted: paraphrase

Sentence 1: Tỷ lệ cao nhất sẽ lên tới 4,45% đối với tất cả cư dân có thu nhập chịu thuế trên 500.000 USD.
Sentence 2: Đối với những cư dân có thu nhập trên 500.000 USD, thuế suất thuế thu nhập sẽ tăng lên 4,45%.
True Label: paraphrase
Predicted: paraphrase

Sentence 1: Cooley cho biết ông mong đợi Muhammad cũng sẽ được gọi làm nhân chứng tương tự tại phiên điều trần trước khi xét xử Malvo.
Sentence 2: Lee Boyd Malvo sẽ được triệu tập làm nhân chứng vào thứ Tư trong phiên điều trần sơ thẩm đối với nghi phạm bắn tỉa John Allen Muhammad.
True Label: not_paraphrase
Predicted: paraphrase

Sentence 1: Theo Bộ Thương mại Hàng hải, con tàu 37 tuổi này được đăng ký với Alpha Shipping Inc. có trụ sở tại quốc gia Quần đảo Marshall ở Thái Bình Dương.
Sentence 2: Baltic Sky l

In [100]:
# Suppose you just computed MNLI accuracy
mrpc_accuracy = compute_accuracy(mrpc_results)

# Append MNLI result to df_results
df_results.loc[len(df_results)] = {
    "task": "MRPC",
    "accuracy": mrpc_accuracy
}

print(df_results)

   task  accuracy
0  CoLA  0.700000
1  MNLI  0.716216
2  MRPC  0.764706


# Load & Train QNLI

In [101]:
train_path = r"ViGLUE_Law\qnli\train_phi4.json"
val_path = r"ViGLUE_Law\qnli\validation_phi4.json"

train_dataset, val_dataset = load_json_dataset(train_path, val_path)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 1479
Validation dataset size: 76


In [102]:
def formatting_prompt_func_QNLI(example):
    return (
        "<Instruct>Does the following sentence answer the question (answer entailment or not_entailment)?</Instruct>\n"
        "<Format>Question-answering format</Format>\n"
        f"<Question>Question: {example['question']}\nSentence: {example['sentence']}</Question>\n"
        f"<Answer>{example['label']}</Answer>"
    )


In [103]:
training_args = SFTConfig(
    output_dir="./sft_output_QNLI",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    logging_steps=10,
    eval_strategy="epoch",
    #save_strategy="steps",               # Save model every 'n' steps instead of epochs
    #save_steps=500,                       # Save every 500 steps (adjust as needed)
    save_total_limit=2,                   # Keep only the last 2 checkpoints
    #load_best_model_at_end=True,          # Load the best model at the end based on eval loss or accuracy
    metric_for_best_model="eval_loss",    # Track accuracy for best model selection
    greater_is_better=True               # Higher accuracy is better
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=formatting_prompt_func_QNLI,
    )

trainer.train()

metrics = trainer.evaluate()

print(metrics)

Applying formatting function to train dataset:   0%|          | 0/1479 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/1479 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1479 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1479 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1479 [00:00<?, ? examples/s]

...

Adding EOS to eval dataset:   0%|          | 0/76 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/76 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/76 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,1.374100,1.484368
2,0.855300,1.650114
3,0.382000,1.952103
4,0.177400,2.127487
5,0.119300,2.201962
6,0.096800,2.230487
7,0.082400,2.267182
8,0.081300,2.293474
9,0.080700,2.313060
10,0.072000,2.334930


{'eval_loss': 2.334929943084717, 'eval_runtime': 6.9505, 'eval_samples_per_second': 10.935, 'eval_steps_per_second': 1.439}


In [104]:
def evaluate_pipeline_on_qnli(val_dataset, max_samples=10):
    results = []

    for i, example in enumerate(val_dataset):
        if i >= max_samples:
            break

        question = example["question"]
        sentence = example["sentence"]
        # Build an MNLI prompt
        prompt = (
            "<Instruct>Does the following sentence answer the question (answer entailment or not_entailment)?</Instruct>\n"
            "<Format>Question-answering format</Format>\n"
            f"<Question>Question: {example['question']}\nSentence: {example['sentence']}</Question>\n"
            f"<Answer>"
    )


        # Generate and extract the model’s answer
        output = inference(prompt, max_new_tokens=10)[0]["generated_text"]
        # Anything after the last <Answer> tag is our prediction
        predicted = output.split("<Answer>")[-1].strip().replace("</Answer>", "").lower()

        results.append({
            "question": question,
            "sentence": sentence,
            "true_label": example["label"].lower(),
            "predicted": predicted
        })

    return results

qnli_results = evaluate_pipeline_on_qnli(val_dataset, max_samples=1000)

for res in qnli_results:
    print(f"Question: {res['question']}")
    print(f"Sentence: {res['sentence']}")
    print(f"True Label: {res['true_label']}")
    print(f"Predicted: {res['predicted']}\n")

Question: Điều gì có hiệu lực sau khi hiến pháp mới được công bố?
Sentence: Kể từ ngày đó, hiến pháp mới báo trước nền Cộng hòa thứ hai có hiệu lực.
True Label: not_entailment
Predicted: entailment

Question: Tòa án Công lý tuyên bố Steymann có quyền gì?
Sentence: Tòa án Công lý cho rằng ông Steymann có quyền ở lại, miễn là có ít nhất một "sự trao đổi gián tiếp" cho công việc ông đã làm.
True Label: not_entailment
Predicted: not_entailment

Question: Điều gì cộng với giá gạo tăng cao đã khiến người nghèo khó hoặc không thể theo kịp?
Sentence: Chính sách thuế bất động sản theo giá trị cộng với giá cả tăng cao đã khiến người dân có thu nhập thấp khó hoặc không thể theo kịp.
True Label: not_entailment
Predicted: not_entailment

Question: Vì sao người phát tờ rơi trong tòa án không bị bắt?
Sentence: Các nhà hoạt động của Hiệp hội Bồi thẩm đoàn được thông tin đầy đủ đôi khi đã phát các tờ rơi giáo dục bên trong các tòa án bất chấp những lời khuyên là không nên làm vậy; theo FIJA, nhiều ngườ

In [105]:
# Suppose you just computed accuracy
qnli_accuracy = compute_accuracy(qnli_results)

# Append MNLI result to df_results
df_results.loc[len(df_results)] = {
    "task": "QNLI",
    "accuracy": qnli_accuracy
}

print(df_results)

   task  accuracy
0  CoLA  0.700000
1  MNLI  0.716216
2  MRPC  0.764706
3  QNLI  0.736842


# Load & Train QQP

In [110]:
from sklearn.model_selection import train_test_split
import json

def load_json_dataset(path, val_size=0.1, seed=42):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Split into train and val
    train_data, val_data = train_test_split(data, test_size=val_size, random_state=seed)

    return train_data, val_data

# Usage
train_path = r"ViGLUE_Law\qqp\train_phi4.json"
train_dataset, val_dataset = load_json_dataset(train_path, val_size=0.1)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")


Train dataset size: 6277
Validation dataset size: 698


In [116]:
from datasets import Dataset

# Assuming train_dataset and val_dataset are currently lists of dicts
train_dataset = Dataset.from_list(train_dataset)
val_dataset = Dataset.from_list(val_dataset)

In [ ]:
def formatting_prompt_func_QQP(example):
    return (
        "<Instruct>Do these two questions convey the same meaning? (answer duplicate or not_duplicate)</Instruct>\n"
        "<Format>Question pair format</Format>\n"
        f"<Question>Question 1: {example['question1']}\nQuestion 2: {example['question2']}</Question>\n"
        f"<Answer>{example['label']}</Answer>"
    )


In [117]:
training_args = SFTConfig(
    output_dir="./sft_output_QQP",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",               # Save model every 'n' steps instead of epochs
    #save_steps=500,                       # Save every 500 steps (adjust as needed)
    save_total_limit=1,                   # Keep only the last 2 checkpoints
    load_best_model_at_end=True,          # Load the best model at the end based on eval loss or accuracy
    metric_for_best_model="eval_loss",    # Track accuracy for best model selection
    greater_is_better= False             # Higher accuracy is better
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=formatting_prompt_func_QQP,
    )

trainer.train()

metrics = trainer.evaluate()

print(metrics)

Applying formatting function to train dataset:   0%|          | 0/6277 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/6277 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/6277 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/6277 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/6277 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/698 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/698 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.922500,0.919388
2,0.654800,0.923706
3,0.360700,1.041565
4,0.192300,1.139127
5,0.147600,1.200042
6,0.130400,1.220566
7,0.121500,1.240579
8,0.116300,1.261057
9,0.112100,1.276177
10,0.106100,1.305982


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


{'eval_loss': 0.9193881154060364, 'eval_runtime': 8.8144, 'eval_samples_per_second': 79.188, 'eval_steps_per_second': 9.984}


In [124]:
# After training QQP…
inference = pipeline(
    "text-generation",
    model=trainer.model,       # now the QQP‐trained model
    tokenizer=trainer.tokenizer,
    device=0
)

def evaluate_pipeline_on_qqp(val_dataset, max_samples=10):
    results = []

    for i, example in enumerate(val_dataset):
        if i >= max_samples:
            break

        question1 = example["question1"]
        question2 = example["question2"]
        # Build an QQP prompt
        prompt = (
            "<Instruct>Do these two questions convey the same meaning? (answer duplicate or not_duplicate)</Instruct>\n"
            "<Format>Question pair format</Format>\n"
            f"<Question>Question 1: {example['question1']}\nQuestion 2: {example['question2']}</Question>\n"
            f"<Answer>"
    )


        # Generate and extract the model’s answer
        output = inference(prompt, max_new_tokens=10)[0]["generated_text"]
        # Anything after the last <Answer> tag is our prediction
        predicted = output.split("<Answer>")[-1].strip().replace("</Answer>", "").lower()

        results.append({
            "question1": question1,
            "question2": question2,
            "true_label": example["label"].lower(),
            "predicted": predicted
        })

    return results

qqp_results = evaluate_pipeline_on_qqp(val_dataset, max_samples=1000)

for res in qqp_results:
    print(f"Question: {res['question1']}")
    print(f"Sentence: {res['question2']}")
    print(f"True Label: {res['true_label']}")
    print(f"Predicted: {res['predicted']}\n")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Device set to use cuda:0


Question: 11 trò chơi điện tử bị cấm mà bạn sẽ không bao giờ chơi được là gì?
Sentence: Những người bảo thủ có chơi trò chơi điện tử không?
True Label: not_duplicate
Predicted: not_duplicate

Question: Luật thực chất và sự khác biệt là luật tố tụng là gì?
Sentence: Sự khác biệt giữa luật tố tụng và luật thực chất là gì?
True Label: duplicate
Predicted: duplicate

Question: Quỹ điện tử và chuyển khoản ngân hàng khác nhau như thế nào?
Sentence: Chuyển khoản điện tử và chuyển khoản ngân hàng khác nhau như thế nào?
True Label: duplicate
Predicted: duplicate

Question: Từ quan điểm công bằng thuần túy, thuế lũy tiến có phải là không công bằng?
Sentence: Thuế lũy tiến có công bằng không?
True Label: duplicate
Predicted: duplicate

Question: Làm thế nào để tôi giải quyết thời gian thông báo không chắc chắn của công ty?
Sentence: Làm cách nào để giải quyết công ty với thời gian thông báo không chắc chắn?
True Label: not_duplicate
Predicted: duplicate

Question: Điều gì sẽ xảy ra nếu tôi không 

In [125]:
# Suppose you just computed accuracy
qqp_accuracy = compute_accuracy(qqp_results)

# Append MNLI result to df_results
df_results.loc[len(df_results)] = {
    "task": "QQP",
    "accuracy": qqp_accuracy
}

print(df_results)

   task  accuracy
0  CoLA  0.700000
1  MNLI  0.716216
2  MRPC  0.764706
3  QNLI  0.736842
4   QQP  0.783668
5   QQP  0.783668


# Load & Train RTE

In [131]:
train_path = r"ViGLUE_Law\rte\train_phi4.json"
val_path = r"ViGLUE_Law\rte\validation_phi4.json"

def load_json_dataset(train_path, val_path):
    dataset = load_dataset('json', data_files={
        'train': train_path,
        'validation': val_path
    })
    return dataset['train'], dataset['validation']

train_dataset, val_dataset = load_json_dataset(train_path, val_path)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 44
Validation dataset size: 3


In [135]:
def formatting_prompt_func_RTE(example):
    return (
        "<Instruct>Does the following sentence entail the hypothesis (answer entailment or not_entailment)?</Instruct>\n"
        "<Format>Premise-hypothesis format</Format>\n"
        f"<Question>Premise: {example['sentence1']}\nHypothesis: {example['sentence2']}</Question>\n"
        f"<Answer>{example['label']}</Answer>"
        )


In [136]:
training_args = SFTConfig(
    output_dir="./sft_output_RTE",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",               # Save model every 'n' steps instead of epochs
    #save_steps=500,                       # Save every 500 steps (adjust as needed)
    save_total_limit=1,                   # Keep only the last 2 checkpoints
    load_best_model_at_end=True,          # Load the best model at the end based on eval loss or accuracy
    metric_for_best_model="eval_loss",    # Track accuracy for best model selection
    greater_is_better= False             # Higher accuracy is better
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=formatting_prompt_func_RTE,
    )

trainer.train()

metrics = trainer.evaluate()

print(metrics)

Applying formatting function to train dataset:   0%|          | 0/44 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/44 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/44 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/44 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/44 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,1.474740
2,No log,1.334262
3,No log,1.321115
4,1.650700,1.372944
5,1.650700,1.442207
6,1.650700,1.515430
7,0.530600,1.580022
8,0.530600,1.624967
9,0.530600,1.652164
10,0.161300,1.661680


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


{'eval_loss': 1.321115493774414, 'eval_runtime': 0.2601, 'eval_samples_per_second': 11.532, 'eval_steps_per_second': 3.844}


In [138]:
def evaluate_pipeline_on_rte(val_dataset, max_samples=10):
    results = []

    for i, example in enumerate(val_dataset):
        if i >= max_samples:
            break

        sentence1 = example["sentence1"]
        sentence2 = example["sentence2"]
        # Build an MNLI prompt
        prompt = (
            "<Instruct>Does the following sentence entail the hypothesis (answer entailment or not_entailment)?</Instruct>\n"
            "<Format>Premise-hypothesis format</Format>\n"
            f"<Question>Premise: {example['sentence1']}\nHypothesis: {example['sentence2']}</Question>\n"
            f"<Answer>"
    )


        # Generate and extract the model’s answer
        output = inference(prompt, max_new_tokens=10)[0]["generated_text"]
        # Anything after the last <Answer> tag is our prediction
        predicted = output.split("<Answer>")[-1].strip().replace("</Answer>", "").lower()

        results.append({
            "sentence1": sentence1,
            "sentence2": sentence2,
            "true_label": example["label"].lower(),
            "predicted": predicted
        })

    return results

rte_results = evaluate_pipeline_on_rte(val_dataset, max_samples=1000)

for res in rte_results:
    print(f"Sentence 1: {res['sentence1']}")
    print(f"Sentence 2: {res['sentence2']}")
    print(f"True Label: {res['true_label']}")
    print(f"Predicted: {res['predicted']}\n")

Sentence 1: Việc kéo dài thời gian độc quyền tiếp thị có hiệu lực đối với thuốc được thiết kế để mang lại cho các công ty dược phẩm một khoản lợi nhuận hợp lý.
Sentence 2: Các loại thuốc triển vọng phải có tuổi thọ lâu dài trên thị trường sau khi được phê duyệt theo quy định để được phát triển.
True Label: entailment
Predicted: entailment

Sentence 1: Phương Tây thích tập trung vào các loài động vật có nguy cơ tuyệt chủng hơn là con người có nguy cơ tuyệt chủng. Voi châu Phi bị những kẻ săn trộm săn lùng, tước ngà và cất giấu. Số lượng ngà voi ở châu Phi giảm từ 1,2 triệu xuống còn 600.000 trong một thập kỷ cho đến khi CITES - Công ước về buôn bán quốc tế các loài có nguy cơ tuyệt chủng - cấm buôn bán ngà voi.
Sentence 2: Một công ước quốc tế cấm buôn bán ngà voi.
True Label: not_entailment
Predicted: not_entailment

Sentence 1: Hình phạt tử hình đóng vai trò như một biện pháp ngăn chặn.
Sentence 2: Hình phạt tử hình là một biện pháp ngăn chặn tội phạm.
True Label: not_entailment
Predi

In [146]:
# Suppose you just computed accuracy
rte_accuracy = compute_accuracy(rte_results)

# Append MNLI result to df_results
df_results.loc[len(df_results)] = {
    "task": "RTE",
    "accuracy": rte_accuracy
}

print(df_results)

   task  accuracy
0  CoLA  0.700000
1  MNLI  0.716216
2  MRPC  0.764706
3  QNLI  0.736842
5   RTE  1.000000


# Load & Train SST2

In [140]:
train_path = r"ViGLUE_Law\sst2\train_phi4.json"
val_path = r"ViGLUE_Law\sst2\validation_phi4.json"

train_dataset, val_dataset = load_json_dataset(train_path, val_path)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 946
Validation dataset size: 5


In [142]:
def formatting_prompt_func_SST2(example):
    return (
         "<Instruct>Is the sentiment of the following sentence positive or negative?</Instruct>\n"
        "<Format>Sentiment classification format</Format>\n"
        f"<Question>Sentence: {example['sentence']}</Question>\n"
        f"<Answer>{example['label']}</Answer>"
)


In [143]:
training_args = SFTConfig(
    output_dir="./sft_output_SST2",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",               # Save model every 'n' steps instead of epochs
    #save_steps=500,                       # Save every 500 steps (adjust as needed)
    save_total_limit=1,                   # Keep only the last 2 checkpoints
    load_best_model_at_end=True,          # Load the best model at the end based on eval loss or accuracy
    metric_for_best_model="eval_loss",    # Track accuracy for best model selection
    greater_is_better= False             # Higher accuracy is better
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=formatting_prompt_func_SST2,
    )

trainer.train()

metrics = trainer.evaluate()

print(metrics)

Applying formatting function to train dataset:   0%|          | 0/946 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/946 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/946 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/946 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/946 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.637200,1.075249
2,0.367800,1.050775
3,0.229200,1.163659
4,0.196200,1.178219
5,0.178100,1.141283
6,0.160700,1.233445
7,0.157100,1.252970
8,0.154400,1.293174
9,0.148700,1.319837
10,0.144500,1.347046


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


{'eval_loss': 1.0507745742797852, 'eval_runtime': 0.0433, 'eval_samples_per_second': 115.537, 'eval_steps_per_second': 23.107}


In [144]:
def evaluate_pipeline_on_sst2(val_dataset, max_samples=10):
    results = []

    for i, example in enumerate(val_dataset):
        if i >= max_samples:
            break

        sentence = example["sentence"]
        # Build an MNLI prompt
        prompt = (
            "<Instruct>Is the sentiment of the following sentence positive or negative?</Instruct>\n"
            "<Format>Sentiment classification format</Format>\n"
            f"<Question>Sentence: {example['sentence']}</Question>\n"
            f"<Answer>"
    )


        # Generate and extract the model’s answer
        output = inference(prompt, max_new_tokens=10)[0]["generated_text"]
        # Anything after the last <Answer> tag is our prediction
        predicted = output.split("<Answer>")[-1].strip().replace("</Answer>", "").lower()

        results.append({
            "sentence": sentence,
            "true_label": example["label"].lower(),
            "predicted": predicted
        })

    return results

sst2_results = evaluate_pipeline_on_sst2(val_dataset, max_samples=1000)

for res in sst2_results:
    print(f"Sentence 1: {res['sentence']}")
    print(f"True Label: {res['true_label']}")
    print(f"Predicted: {res['predicted']}\n")

Sentence 1: hoặc đóng thuế năm ngoái với vợ cũ của bạn .
True Label: negative
Predicted: negative

Sentence 1: phải có giấy phép đạo diễn thì Ed Burns mới có thể bị thu hồi.
True Label: negative
Predicted: negative

Sentence 1: đó là một trò lừa đảo
True Label: negative
Predicted: negative

Sentence 1: đó là một sự lôi kéo có lỗi .
True Label: negative
Predicted: negative

Sentence 1: khiếu châm biếm lành mạnh của anh ấy nhẹ nhàng và vui vẻ ...
True Label: positive
Predicted: positive



In [ ]:
# Suppose you just computed accuracy
sst2_accuracy = compute_accuracy(sst2_results)

# Append MNLI result to df_results
df_results.loc[len(df_results)] = {
    "task": "SST2",
    "accuracy": sst2_accuracy
}

print(df_results)

5


# Load & Train WNLI 

In [156]:
train_path = r"ViGLUE_Law\wnli\train_phi4.json"
val_path = r"ViGLUE_Law\wnli\validation_phi4.json"

train_dataset, val_dataset = load_json_dataset(train_path, val_path)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 7
Validation dataset size: 3


In [157]:
def formatting_prompt_func_WNLI(example):
    return (
        "<Instruct>Does the following sentence answer the question (answer entailment or not_entailment)?</Instruct>\n"
        "<Format>Entailment format</Format>\n"
        f"<Question>Assume that the following is true: {example['sentence1']}\n"
        f"Does this mean that {example['sentence2']}?</Question>\n"
        f"<Answer>{example['label']}</Answer>"
    )


In [158]:
training_args = SFTConfig(
    output_dir="./sft_output_WNLI",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",               # Save model every 'n' steps instead of epochs
    #save_steps=500,                       # Save every 500 steps (adjust as needed)
    save_total_limit=1,                   # Keep only the last 2 checkpoints
    load_best_model_at_end=True,          # Load the best model at the end based on eval loss or accuracy
    metric_for_best_model="eval_loss",    # Track accuracy for best model selection
    greater_is_better= False             # Higher accuracy is better
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=formatting_prompt_func_WNLI,
    )

trainer.train()

metrics = trainer.evaluate()

print(metrics)

Applying formatting function to train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/7 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,2.026040
2,No log,1.330089
3,No log,0.933265
4,No log,0.725073
5,No log,0.595042
6,No log,0.517802
7,No log,0.472395
8,No log,0.445836
9,No log,0.432409
10,0.774500,0.427350


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


{'eval_loss': 0.4273499548435211, 'eval_runtime': 0.043, 'eval_samples_per_second': 69.781, 'eval_steps_per_second': 23.26}


In [159]:
def evaluate_pipeline_on_wnli(val_dataset, max_samples=10):
    results = []

    for i, example in enumerate(val_dataset):
        if i >= max_samples:
            break

        sentence1 = example["sentence1"]
        sentence2 = example["sentence2"]
        # Build an MNLI prompt
        prompt = (
            "<Instruct>Does the following sentence answer the question (answer entailment or not_entailment)?</Instruct>\n"
            "<Format>Entailment format</Format>\n"
            f"<Question>Assume that the following is true: {example['sentence1']}\n"
            f"Does this mean that {example['sentence2']}?</Question>\n"
            f"<Answer>"
    )


        # Generate and extract the model’s answer
        output = inference(prompt, max_new_tokens=10)[0]["generated_text"]
        # Anything after the last <Answer> tag is our prediction
        predicted = output.split("<Answer>")[-1].strip().replace("</Answer>", "").lower()

        results.append({
            "sentence1": sentence1,
            "sentence2": sentence2,
            "true_label": example["label"].lower(),
            "predicted": predicted
        })

    return results

wnli_results = evaluate_pipeline_on_wnli(val_dataset, max_samples=1000)

for res in wnli_results:
    print(f"Sentence 1: {res['sentence1']}")
    print(f"Sentence 2: {res['sentence2']}")
    print(f"True Label: {res['true_label']}")
    print(f"Predicted: {res['predicted']}\n")

Sentence 1: Bernard, người đã không nói với quan chức chính phủ rằng anh ta chưa đủ 21 tuổi khi nộp đơn yêu cầu bồi thường nhà ở, không cho rằng mình đã làm bất cứ điều gì không trung thực. Tuy nhiên, bất cứ ai biết rằng anh ta 19 tuổi đều có thể tước bỏ quyền lợi của anh ta.
Sentence 2: Bernard chưa đầy 21 tuổi khi nộp đơn yêu cầu bồi thường nhà ở.
True Label: entailment
Predicted: entailment

Sentence 1: Các ủy viên hội đồng thành phố đã từ chối cấp phép cho những người biểu tình vì họ ủng hộ bạo lực.
Sentence 2: Những người biểu tình ủng hộ bạo lực.
True Label: entailment
Predicted: entailment

Sentence 1: Cảnh sát đã bắt giữ tất cả các thành viên của băng đảng. Họ đang cố gắng điều hành việc buôn bán ma túy trong khu phố.
Sentence 2: Cảnh sát đang cố gắng điều hành hoạt động buôn bán ma túy trong khu vực lân cận.
True Label: not_entailment
Predicted: entailment



In [160]:
# Suppose you just computed accuracy
wnli_accuracy = compute_accuracy(wnli_results)

# Append MNLI result to df_results
df_results.loc[len(df_results)] = {
    "task": "WNLI",
    "accuracy": wnli_accuracy
}

print(df_results)

   task  accuracy
0  CoLA  0.700000
1  MNLI  0.716216
2  MRPC  0.764706
3  QNLI  0.736842
4  SST2  1.000000
5   RTE  1.000000
6  WNLI  0.666667
